In [65]:
import heapq
import time
import pandas as pd
import torch
from torch_geometric.data import Data
import os
from glob import glob
import numpy as np

from thesis_floor_halkes.features.dynamic.getter import DynamicFeatureGetterDataFrame
from thesis_floor_halkes.features.graph.graph_generator import create_osmnx_sub_graph_only_inside_helmond, get_edge_features_subgraph, get_node_features_subgraph, plot_sub_graph_in_and_out_nodes_helmond
from thesis_floor_halkes.features.static.new_getter import get_static_data_object_subgraph
from thesis_floor_halkes.utils.adj_matrix import build_adjecency_matrix
from thesis_floor_halkes.utils.haversine import haversine

In [4]:
from thesis_floor_halkes.benchmarks.time_dependent_A_star import time_dependent_a_star
from thesis_floor_halkes.benchmarks.time_dependent_dijkstra import time_dependent_dijkstra

In [7]:
static_data = torch.load('/home/floor/projects/FINAL/thesis_ambulance_optimization/data/training_data/network_0/network_0_29_15.pt', weights_only=False)

In [8]:
static_data

Data(x=[178, 4], edge_index=[2, 526], edge_attr=[526, 2], start_node=29, end_node=15, G_sub=MultiDiGraph with 178 nodes and 526 edges, G_pt=MultiDiGraph with 178 nodes and 566 edges, timeseries=        node_id osmid_original tlc_name        lat       lon  has_light  \
0             0     [42728835]        0  51.475534  5.678364          0   
1             0     [42728835]        0  51.475534  5.678364          0   
2             0     [42728835]        0  51.475534  5.678364          0   
3             0     [42728835]        0  51.475534  5.678364          0   
4             0     [42728835]        0  51.475534  5.678364          0   
...         ...            ...      ...        ...       ...        ...   
256315      177     [42762652]        0  51.486164  5.670416          0   
256316      177     [42762652]        0  51.486164  5.670416          0   
256317      177     [42762652]        0  51.486164  5.670416          0   
256318      177     [42762652]        0  51.486164  5.67

In [20]:
dynamic_node_idx = {
    "status": 0,
    "wait_time": 1,
    "current_node": 2,
    "visited_nodes": 3,
}

static_node_idx = {
    "lat": 0,
    "lon": 1,
    "has_light": 2,
    "dist_to_goal": 3,
}

static_edge_idx = {
    "length": 0,
    "speed": 1,
}

dynamic_feature_getter = DynamicFeatureGetterDataFrame()

In [48]:
def time_dependent_dijkstra(
    static_data, 
    dynamic_feature_getter, 
    dynamic_node_idx,
    static_node_idx,
    static_edge_idx,
):
    df = static_data.timeseries.copy()
    start_node = static_data.start_node
    
    df_start = df[df['node_id'] == start_node]
    time_stamps = sorted(df_start["timestamp"].unique())
    ts0 = pd.to_datetime(time_stamps[0])  # ensures it's a pandas.Timestamp
    t0 = time_stamps.index(ts0) # index of the first timestamp
    
    T = len(time_stamps)
    
    adj = build_adjecency_matrix(static_data.num_nodes, static_data)
    start = static_data.start_node
    end = static_data.end_node
    
    class _TmpEnv:
        pass 
    tmp = _TmpEnv()
    tmp.static_data = static_data
    tmp.static_node_idx = static_node_idx
    
    heap = [(0.0, start, t0, [start])]
    best = {(start, t0): 0.0}
    
    while heap: 
        cost, node, t_idx, path = heapq.heappop(heap)
        # print(f"[POP] cost={cost:.3f}, node={node}, t_idx={t_idx}, path={path}")
        if cost > best.get((node, t_idx), float("inf")):
            # print("       → skipping stale entry")
            continue
        if node == end:
            # print(f"[FOUND] cost={cost:.3f}, path={path}")
            return cost, path
        next_t = t_idx + 1
        if next_t >= T:
            # print(f"       → next_t={next_t} ≥ T={T}, no further expansion")
            continue
        # print(f"       → expanding at time index {next_t} (ts={time_stamps[next_t]})")
    
        dyn = dynamic_feature_getter.get_dynamic_features(
            environment = tmp, 
            traffic_light_idx = static_node_idx["has_light"],
            current_node = node, 
            visited_nodes = path,
            time_step = next_t, 
            sub_node_df = df,
        )
        wait_times = dyn.x[:, dynamic_node_idx["wait_time"]]
        
        for nbr, eidx in adj[node]:
            length = static_data.edge_attr[eidx, static_edge_idx["length"]]
            speed = static_data.edge_attr[eidx, static_edge_idx["speed"]]
            travel_time = length / (speed / 3.6)
            light_status = dyn.x[nbr, dynamic_node_idx["status"]].item()
            if light_status == 1:  # green light
                wait = 0.0
            else:  # red light
                wait = wait_times[nbr].item()
            
            ts = time_stamps[next_t]
            orig_wait_series = df[
                (df["node_id"] == nbr) & (df["timestamp"] == ts)
            ]
            # print(f"  Original wait series:\n {orig_wait_series}\n")
            # print(f"node = {nbr} @ ts={ts}"
            #       f"> dyn.get() = {wait:.2f}s ")
            
            
            new_cost = cost + travel_time + wait
            
            # print(
            #         f"         edge {node}→{nbr}: travel={travel_time:.2f}, "
            #         f"wait={wait:.2f} → new_cost={new_cost:.2f}"
            #     )
            
            key = (nbr, next_t)
            if new_cost < best.get(key, float("inf")):
                best[key] = new_cost
                heapq.heappush(heap, (new_cost, nbr, next_t, path + [nbr]))
                # print(f"           • pushed (cost={new_cost:.2f}, node={nbr}, t={next_t})")
        
    return None, float("inf")  # No path found
    

In [ ]:
def run_dijkstra_on_dataset(split_name, base_dir):
    print(f"\nRunning Time-Dependent Dijkstra on {split_name.upper()} set")
    results = []

    network_dirs = sorted([
        os.path.join(base_dir, d)
        for d in os.listdir(base_dir)
        if os.path.isdir(os.path.join(base_dir, d))
    ])

    for network_dir in network_dirs:
        pt_files = glob(os.path.join(network_dir, "*.pt"))
        print(f"Found {len(pt_files)} graphs in {network_dir}")

        for pt_file in pt_files:
            try:
                static_data = torch.load(pt_file, weights_only=False)

                dynamic_feature_getter = DynamicFeatureGetterDataFrame()

                start = time.time()
                cost, route = time_dependent_dijkstra(
                    static_data,
                    dynamic_feature_getter,
                    dynamic_node_idx,
                    static_node_idx,
                    static_edge_idx,
                )
                end = time.time()

                results.append({
                    "file": pt_file,
                    "network": f"{os.path.basename(network_dir)}_{static_data.start_node}_{static_data.end_node}",
                    "success": cost is not None,
                    "cost": cost.item() if cost is not None else float("inf"),
                    "route": route,
                    "route_len": len(route),
                    "runtime_s": end - start,
                    "split": split_name
                })
            except Exception as e:
                print(f"Error in file {pt_file}: {e}")
                results.append({
                    "file": pt_file,
                    "network": os.path.basename(network_dir),
                    "success": False,
                    "cost": float("inf"),
                    "route_len": 0,
                    "runtime_s": 0.0,
                    "split": split_name
                })
        break

    return pd.DataFrame(results)


In [ ]:
df = run_dijkstra_on_dataset("train", "/home/floor/projects/FINAL/thesis_ambulance_optimization/data/training_data")


Running Time-Dependent Dijkstra on TRAIN set
Found 4 graphs in /home/floor/projects/FINAL/thesis_ambulance_optimization/data/training_data/network_0


In [54]:
df

,file,network,success,cost,route,route_len,runtime_s,split
0,/home/floor/projects/FINAL/thesis_ambulance_op...,network_0_51_15,True,200.515228,"[51, 73, 64, 59, 49, 40, 30, 38, 16, 14, 15]",11,14.358259,train
1,/home/floor/projects/FINAL/thesis_ambulance_op...,network_0_29_15,True,186.441376,"[29, 28, 22, 17, 19, 20, 30, 38, 16, 14, 15]",11,11.472449,train
2,/home/floor/projects/FINAL/thesis_ambulance_op...,network_0_137_167,True,71.614182,"[137, 143, 157, 150, 167]",5,0.562073,train
3,/home/floor/projects/FINAL/thesis_ambulance_op...,network_0_107_47,True,244.224152,"[107, 105, 103, 51, 73, 64, 59, 49, 40, 56, 65...",12,18.279907,train


In [68]:
def calculate_success_travel_time_score(df, alpha, beta, min_time=0, max_time=1000):
    """
    Calculate a normalized score from individual route success and travel times.
    
    - `alpha`: weight for success rate
    - `beta`: weight for normalized penalized travel time
    - `min_time`, `max_time`: used to normalize travel times
    """
    # Extract per-sample success (1.0/0.0) and travel time
    success_rates = df['success'].astype(float).tolist()
    travel_times = df['cost'].tolist()

    # Min-max normalize travel times
    penalized_travel_times = np.array(travel_times)
    penalized_travel_times = (penalized_travel_times - min_time) / (max_time - min_time)
    penalized_travel_times = np.clip(penalized_travel_times, 0, 1)

    # Compute final score
    score = (
        alpha * np.mean(success_rates)
        - beta * np.mean(penalized_travel_times)
    )
    
    return score.item()

In [69]:
calculate_success_travel_time_score(df, alpha=0.8, beta=0.2)

0.7648602531433106

In [ ]:
def time_dependent_a_star(
    static_data,
    dynamic_feature_getter,
    dynamic_node_idx,
    static_node_idx,
    static_edge_idx,
):
    df = static_data.timeseries.copy()
    start_node = static_data.start_node
    end_node = static_data.end_node
    # print(f"Start node: {start_node}")
    # print(f"End node: {end_node}")

    df_start = df[df['node_id'] == start_node]
    time_stamps = sorted(df_start["timestamp"].unique())
    ts0 = pd.to_datetime(time_stamps[0])  # ensures it's a pandas.Timestamp
    t0 = time_stamps.index(ts0) # index of the first timestamp
    
    T = len(time_stamps)

    adj = build_adjecency_matrix(static_data.num_nodes, static_data)

    # Heuristic: straight-line distance to goal / max speed (in seconds)
    max_speed = static_data.edge_attr[:, static_edge_idx['speed']].max().item()
    dist_to_goal = static_data.x[:, static_node_idx['dist_to_goal']]  # meters

    def heuristic(node):
        # max_speed in km/h, convert to m/s
        return dist_to_goal[node].item() / (max_speed / 3.6 + 1e-6)

    class _TmpEnv:
        pass
    tmp = _TmpEnv()
    tmp.static_data = static_data
    tmp.static_node_idx = static_node_idx

    heap = [(heuristic(start_node), 0.0, start_node, t0, [start_node])]
    best = {(start_node, t0): 0.0}

    while heap:
        est_total, cost, node, t_idx, path = heapq.heappop(heap)
        if cost > best.get((node, t_idx), float("inf")):
            continue
        if node == end_node:
            # print(f"[FOUND] cost={cost:.3f}, path={path}")
            return cost, path
        next_t = t_idx + 1
        if next_t >= T:
            continue

        dyn = dynamic_feature_getter.get_dynamic_features(
            environment=tmp,
            traffic_light_idx=static_node_idx["has_light"],
            current_node=node,
            visited_nodes=path,
            time_step=next_t,
            sub_node_df=df,
        )
        wait_times = dyn.x[:, dynamic_node_idx["wait_time"]]

        for nbr, eidx in adj[node]:
            length = static_data.edge_attr[eidx, static_edge_idx["length"]]
            speed = static_data.edge_attr[eidx, static_edge_idx["speed"]]
            travel_time = length / (speed / 3.6)
            
            light_status = dyn.x[nbr, dynamic_node_idx["status"]].item()
            if light_status == 1:  # green light
                wait = 0.0
            else:  # red light
                wait = wait_times[nbr].item()
            
            new_cost = cost + travel_time + wait
            key = (nbr, next_t)
            
            if new_cost < best.get(key, float("inf")):
                best[key] = new_cost
                est = new_cost + heuristic(nbr)
                heapq.heappush(heap, (est, new_cost, nbr, next_t, path + [nbr]))

    return None, float("inf")  # No path found


In [72]:
time_dependent_a_star(static_data,dynamic_feature_getter, dynamic_node_idx, static_node_idx, static_edge_idx)

Start node: 29
End node: 15
[FOUND] cost=186.441, path=[29, 28, 22, 17, 19, 20, 30, 38, 16, 14, 15]


(tensor(186.4414), [29, 28, 22, 17, 19, 20, 30, 38, 16, 14, 15])

In [ ]:
def run_on_dataset(split_name, base_dir, routing_fn):
    """
    Run a given routing function (e.g. time_dependent_dijkstra or a_star) 
    on all .pt graph files in the specified split directory.
    
    Parameters:
        split_name (str): "train", "val", or "test"
        base_dir (str): base directory for the split (e.g. "training_data")
        routing_fn (function): the routing function to apply to each graph
    """
    print(f"\nRunning {routing_fn.__name__} on {split_name.upper()} set")
    results = []

    network_dirs = sorted([
        os.path.join(base_dir, d)
        for d in os.listdir(base_dir)
        if os.path.isdir(os.path.join(base_dir, d))
    ])

    for network_dir in network_dirs:
        pt_files = glob(os.path.join(network_dir, "*.pt"))
        print(f"Found {len(pt_files)} graphs in {network_dir}")

        for pt_file in pt_files:
            try:
                static_data = torch.load(pt_file, weights_only=False)
                dynamic_feature_getter = DynamicFeatureGetterDataFrame()

                start = time.time()
                cost, route = routing_fn(
                    static_data,
                    dynamic_feature_getter,
                    dynamic_node_idx,
                    static_node_idx,
                    static_edge_idx,
                )
                end = time.time()

                results.append({
                    "file": pt_file,
                    "network": os.path.basename(network_dir),
                    "success": cost is not None,
                    "cost": cost.item() if cost is not None else float("inf"),
                    "route": route,
                    "route_len": len(route),
                    "runtime_s": end - start,
                    "split": split_name,
                    "algorithm": routing_fn.__name__
                })
            except Exception as e:
                print(f"Error in file {pt_file}: {e}")
                results.append({
                    "file": pt_file,
                    "network": os.path.basename(network_dir),
                    "success": False,
                    "cost": float("inf"),
                    "route_len": 0,
                    "runtime_s": 0.0,
                    "split": split_name,
                    "algorithm": routing_fn.__name__
                })
        break  # Only process the first network for now

    return pd.DataFrame(results)


In [81]:
def run_all_evaluations(base_dirs, output_root, algorithms):
    """
    Runs each algorithm on each dataset split and saves the results.
    
    Parameters:
        base_dirs (dict): {split_name: path_to_split}
                          e.g., {"train": "training_data", "val": "val_data", "test": "test_data"}
        output_root (str): Base output directory (results will go into subfolders here)
        algorithms (dict): {name: function} — e.g., {"dijkstra": time_dependent_dijkstra, "astar": time_dependent_astar}
    """
    os.makedirs(output_root, exist_ok=True)

    for algo_name, algo_fn in algorithms.items():
        algo_output_dir = os.path.join(output_root, f"results_{algo_name}")
        os.makedirs(algo_output_dir, exist_ok=True)

        print(f"\n=== Running algorithm: {algo_name.upper()} ===")
        for split_name, split_dir in base_dirs.items():
            print(f"\n--- Split: {split_name.upper()} ---")
            df = run_on_dataset(split_name, split_dir, routing_fn=algo_fn)

            output_path = os.path.join(algo_output_dir, f"{split_name}.csv")
            df.to_csv(output_path, index=False)
            print(f"Saved {split_name} results to {output_path}")


In [82]:
base_dirs = {
    "train": "/home/floor/projects/FINAL/thesis_ambulance_optimization/data/training_data",
    # "val": "val_data",
    # "test": "test_data"
}

algorithms = {
    "dijkstra": time_dependent_dijkstra,
    "astar": time_dependent_a_star
}

run_all_evaluations(base_dirs, output_root="evaluation_results", algorithms=algorithms)


=== Running algorithm: DIJKSTRA ===

--- Split: TRAIN ---

Running time_dependent_dijkstra on TRAIN set
Found 4 graphs in /home/floor/projects/FINAL/thesis_ambulance_optimization/data/training_data/network_0
Saved train results to evaluation_results/results_dijkstra/train.csv

=== Running algorithm: ASTAR ===

--- Split: TRAIN ---

Running time_dependent_a_star on TRAIN set
Found 4 graphs in /home/floor/projects/FINAL/thesis_ambulance_optimization/data/training_data/network_0
Start node: 51
End node: 15
[FOUND] cost=200.515, path=[51, 73, 64, 59, 49, 40, 30, 38, 16, 14, 15]
Start node: 29
End node: 15
[FOUND] cost=186.441, path=[29, 28, 22, 17, 19, 20, 30, 38, 16, 14, 15]
Start node: 137
End node: 167
[FOUND] cost=71.614, path=[137, 143, 157, 150, 167]
Start node: 107
End node: 47
[FOUND] cost=244.224, path=[107, 105, 103, 51, 73, 64, 59, 49, 40, 56, 65, 47]
Saved train results to evaluation_results/results_astar/train.csv
